### Comparing Models and Vectorization Strategies for Text Classification

This try-it focuses on weighing the positives and negatives of different estimators and vectorization strategies for a text classification problem.  In order to consider each of these components, you should make use of the `Pipeline` and `GridSearchCV` objects in scikitlearn to try different combinations of vectorizers with different estimators.  For each of these, you also want to use the `.cv_results_` to examine the time for the estimator to fit the data.

### The Data

The dataset below is from [kaggle]() and contains a dataset named the "ColBert Dataset" created for this [paper](https://arxiv.org/pdf/2004.12765.pdf).  You are to use the text column to classify whether or not the text was humorous.  It is loaded and displayed below.

**Note:** The original dataset contains 200K rows of data. It is best to try to use the full dtaset. If the original dataset is too large for your computer, please use the 'dataset-minimal.csv', which has been reduced to 100K.

In [19]:
import pandas as pd

In [21]:
df = pd.read_csv('text_data/dataset.csv')

In [23]:
df.head()

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False


#### Task


**Text preprocessing:** As a pre-processing step, perform both `stemming` and `lemmatizing` to normalize your text before classifying. For each technique use both the `CountVectorize`r and `TfidifVectorizer` and use options for stop words and max features to prepare the text data for your estimator.

**Classification:** Once you have prepared the text data with stemming lemmatizing techniques, consider `LogisticRegression`, `DecisionTreeClassifier`, and `MultinomialNB` as classification algorithms for the data. Compare their performance in terms of accuracy and speed.

Share the results of your best classifier in the form of a table with the best version of each estimator, a dictionary of the best parameters and the best score.

In [26]:
pd.DataFrame({'model': ['Logistic', 'Decision Tree', 'Bayes'], 
             'best_params': ['', '', ''],
             'best_score': ['', '', '']}).set_index('model')

,best_params,best_score
model,,
Logistic,,
Decision Tree,,
Bayes,,


In [28]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import nltk

In [30]:
# Download necessary NLTK data
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/odeanmaye/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/odeanmaye/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Preprocessing function
def preprocess_text(text, method="stem"):
    tokens = text.split()
    if method == "stem":
        return " ".join([stemmer.stem(word) for word in tokens])
    elif method == "lemma":
        return " ".join([lemmatizer.lemmatize(word) for word in tokens])

# Apply stemming and lemmatizing
df['text_stemmed'] = df['text'].apply(lambda x: preprocess_text(x, method="stem"))
df['text_lemmatized'] = df['text'].apply(lambda x: preprocess_text(x, method="lemma"))

In [33]:
# Set up training data
X_stemmed = df['text_stemmed']
X_lemmatized = df['text_lemmatized']
y = df['humor']

X_train_stem, X_test_stem, y_train, y_test = train_test_split(X_stemmed, y, test_size=0.2, random_state=42)
X_train_lemma, X_test_lemma, _, _ = train_test_split(X_lemmatized, y, test_size=0.2, random_state=42)

In [34]:
# Define vectorizers with stop words and max features
vectorizers = {
    'count': CountVectorizer(stop_words='english', max_features=500),
    'tfidf': TfidfVectorizer(stop_words='english', max_features=500)
}

# Define classifiers
classifiers = {
    'Logistic': LogisticRegression(max_iter=200),
    'Decision Tree': DecisionTreeClassifier(),
    'Bayes': MultinomialNB()
}

In [35]:
# Function to run classification and capture best results
def evaluate_models(X_train, X_test, y_train, y_test):
    results = {'model': [], 'best_params': [], 'best_score': []}
    for vect_name, vect in vectorizers.items():
        for clf_name, clf in classifiers.items():
            pipeline = Pipeline([
                ('vectorizer', vect),
                ('classifier', clf)
            ])
            # Use GridSearch for parameter tuning
            param_grid = {
                'classifier__C': [0.1, 1, 10] if clf_name == 'Logistic' else None,
                'classifier__max_depth': [10, None] if clf_name == 'Decision Tree' else None,
            }
            grid_search = GridSearchCV(pipeline, param_grid={k: v for k, v in param_grid.items() if v is not None}, scoring='accuracy', n_jobs=-1)
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            score = accuracy_score(y_test, best_model.predict(X_test))
            results['model'].append(f"{clf_name} ({vect_name})")
            results['best_params'].append(grid_search.best_params_)
            results['best_score'].append(score)

    return pd.DataFrame(results).set_index('model')

In [36]:
# Run evaluations
stem_results = evaluate_models(X_train_stem, X_test_stem, y_train, y_test)
lemma_results = evaluate_models(X_train_lemma, X_test_lemma, y_train, y_test)

In [37]:
# Display the best results
best_results = pd.concat([stem_results, lemma_results]).sort_values(by='best_score', ascending=False)
best_results

,best_params,best_score
model,,
Logistic (count),{'classifier__C': 1},0.807600
Logistic (tfidf),{'classifier__C': 1},0.806725
Logistic (count),{'classifier__C': 1},0.802175
Logistic (tfidf),{'classifier__C': 1},0.801225
Bayes (count),{},0.801050
Bayes (tfidf),{},0.799650
Bayes (count),{},0.793050
Bayes (tfidf),{},0.790700
Decision Tree (tfidf),{'classifier__max_depth': None},0.778875
